In [0]:
import numpy as np
import pandas as pd
import cv2

from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import categorical_crossentropy
from keras.layers import BatchNormalization
from keras.layers.convolutional import *
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.utils import np_utils

K.set_image_dim_ordering('tf')
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

import itertools

from PIL import Image
import os, sys
import zipfile
import io

from google.colab import files

np.random.seed(7)

In [0]:
uploaded = files.upload()

In [0]:
Data = zipfile.ZipFile(io.BytesIO(uploaded['Data.zip']), 'r')
Data.extractall()

In [0]:
Data = []
labels_Data = []
for i in range(10):
    xData = str('Data/'+str(i)+'/')
    drData = os.listdir(xData) # dir is your directory path
    number_files = len(drData)
    for names in (drData):
        xData1 = str(xData+names)
        Data.append(xData1)
        labels_Data.append(i)

In [0]:
Data_tuples = []
Data_tuples = list(zip(Data, labels_Data))

In [0]:
Data_df = pd.DataFrame(Data_tuples, columns=['Image_Path','Label'])
Data_df = Data_df.sample(frac=1).reset_index(drop=True)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(Data_df['Image_Path'], Data_df['Label'], test_size=0.33, random_state=42)

In [0]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
Y_train = Y_train.reset_index(drop=True)
Y_test = Y_test.reset_index(drop=True)

In [0]:
t = cv2.imread(Data_df['Image_Path'][3])

In [0]:
image_X_train = []
for i in range(len(X_train)):
    image = cv2.imread(X_train[i])
    image = image/255
    image_X_train.append(image)
image_X_train = np.array(image_X_train)

In [0]:
image_X_test = []
for i in range(len(X_test)):
    image = cv2.imread(X_test[i])
    image = image/255
    image_X_test.append(image)
image_X_test = np.array(image_X_test)

In [0]:
image_X_train.shape[1:]

In [0]:
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

In [0]:
def create_model():    
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(28,28,3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64,(3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [0]:
def train_and_evaluate__model(model, data_train, labels_train, data_test, labels_test):
    hist = model.fit(data_train, labels_train, validation_data=(data_test, labels_test), epochs=100, batch_size=1000, verbose=1)
    return hist
    
Ytst = np.argmax(Y_test,axis=1)
Ytrn = np.argmax(Y_train,axis=1)
# Instantiate the cross validator
skf = StratifiedKFold(n_splits=10, shuffle=True)
# Loop through the indices the split() method returns
for index, (train_indices, val_indices) in enumerate(skf.split(image_X_train, Ytrn)):
    print ("Training on fold " + str(index+1) + "/10...")
    # Generate batches from indices
    xtrain, xval = image_X_train[train_indices], image_X_train[val_indices]
    ytrain, yval = Y_train[train_indices], Y_train[val_indices]
    # Clear model, and create it
    model = None
    model = create_model()
    
    # Debug message I guess
    # print "Training new iteration on " + str(xtrain.shape[0]) + " training samples, " + str(xval.shape[0]) + " validation samples, this may be a while..."
    
    hist = train_and_evaluate__model(create_model(), image_X_train, Y_train, image_X_test, Y_test)
    accuracy_history = hist.history['acc']
    val_accuracy_history = hist.history['val_acc']
    print ("Last training accuracy: " + str(accuracy_history[-1]) + ", last validation accuracy: " + str(val_accuracy_history[-1]))

In [0]:
pred = model.predict(image_X_test)
predicted_class_indices=np.argmax(pred,axis=1)
Y = np.argmax(Y_test,axis=1)

In [0]:
results = []
results = pd.DataFrame(results)

results['Original_Labels'] = Y
results['Predicted_Labels'] = predicted_class_indices

results

In [0]:
scores = model.evaluate(image_X_test, Y_test, verbose=0)
print("CNN Accuracy: %.2f%%" % (scores[1]*100))